In [57]:
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np
import h2o
from h2o.automl import H2OAutoML

X=pd.read_csv('../data/processed/X.csv')
Y=pd.read_csv('../data/processed/Y.csv')
X_predict=pd.read_csv('../data/processed/X_predict.csv')
df_predict=pd.read_csv('../data/processed/df_predict.csv')

In [58]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.1-internal" 2018-10-16; OpenJDK Runtime Environment (build 11.0.1-internal+0-adhoc..src); OpenJDK 64-Bit Server VM (build 11.0.1-internal+0-adhoc..src, mixed mode)
  Starting server from /home/carpiero/miniconda3/envs/ml_env/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpm4h94pqj
  JVM stdout: /tmp/tmpm4h94pqj/h2o_carpiero_started_from_python.out
  JVM stderr: /tmp/tmpm4h94pqj/h2o_carpiero_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Europe/Madrid
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.1.1
H2O_cluster_version_age:,1 month
H2O_cluster_name:,H2O_from_python_carpiero_qcrt4z
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.686 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"accepting new members, healthy"


In [59]:
X_Y=pd.concat([X,Y], axis=1)


In [60]:
x= X.columns.to_list()
y = "price"
x

['carat',
 'depth',
 'table',
 'x',
 'y',
 'z',
 'cut_int',
 'color_int',
 'clarity_int',
 'raw_density']

In [61]:
# H2OAutoML(max_models=50,sort_metric='RMSE') 511.999	
# H2OAutoML(max_models=100,sort_metric='RMSE')    
# H2OAutoML(max_models=200,sort_metric='RMSE')
# H2OAutoML(max_models=400,max_runtime_secs=1000,sort_metric='RMSE')  516.046
# H2OAutoML(max_models=50,max_runtime_secs=1000,sort_metric='RMSE') 512.375
# H2OAutoML(max_models=20,max_runtime_secs=1000,sort_metric='RMSE') 513,651

train_X_Y=h2o.H2OFrame(X_Y)
aml = H2OAutoML(max_models=100,sort_metric='RMSE')
aml.train(x=x, y=y, training_frame=train_X_Y)


Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |██████████████████████████████ (cancelled)


H2OJobCancelled: Job<$03017f00000132d4ffffffff$_ad064a39d583f5c14cf260c9fd934172> was cancelled by the user.

In [ ]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

In [53]:
predict_X=h2o.H2OFrame(X_predict)
predictions_competition= aml.leader.predict(predict_X)
predictions_competition=predictions_competition.as_data_frame()


Parse progress: |█████████████████████████████████████████████████████████| 100%
stackedensemble prediction progress: |████████████████████████████████████| 100%


In [54]:

submission = pd.DataFrame({'id': df_predict['id'], 'price': predictions_competition['predict']})
submission['id'] = submission['id'].astype('int64')
# submission.loc[94,'price'] =20631
submission.to_csv(f'/home/carpiero/IRONHACK/kaggle/submission_H2OAutoML(max_models=40,max_runtime_secs=1000_514_349.csv', index=False)



In [55]:
h2o.cluster().shutdown()

H2O session _sid_ba7f closed.


In [56]:
# H2OAutoML(max_models=20,max_runtime_secs=1000,sort_metric='RMSE')  511.791
# count    13485.000000
# mean      3947.534876
# std       3940.890099
# min        342.779315
# 25%        935.550702
# 50%       2449.032989
# 75%       5329.584411
# max      18128.114212
# Name: price, dtype: float64

# H2OAutoML(max_models=20,max_runtime_secs=1000,sort_metric='RMSE')  514.349
# count    13485.000000
# mean      3945.590390
# std       3957.416982
# min        332.303338
# 25%        929.447763
# 50%       2447.357701
# 75%       5315.242484
# max      18172.341788
# Name: price, dtype: float64


submission['price'].describe()

count    13485.000000
mean      3945.590390
std       3957.416982
min        332.303338
25%        929.447763
50%       2447.357701
75%       5315.242484
max      18172.341788
Name: price, dtype: float64